# Pronóstico de la evolución de pacientes con diabetes

## Descripción del problema real

Los tratamientos médicos están basados en las expectativas de recuperación o el avance de una enfermedad para tomar decisiones. En este caso, un equipo médico desea contar con pronósticos de pacientes con diabetes para tomar decisiones sobre su tratamiento.

## Descripción del problema en términos de los datos

Se desea determinar el progreso de la diabeteis un año hacia adelante a partir de las variables medidas para 442 pacientes. La información está almacenada en el archivo `datos/diabetes.csv`. Las variables medidas son: edad, sexo, indice de masa corporal, presión sanguinea y seis medidas de serum en la sangre. Se desea pronósticar el progreso de la enfermedad a partir de las variables dadas.

## Aproximaciones posibles

En este caso, se desea comparar los resultados de un modelo de regresión lineal y un modelo de redes neuronales artificiales.

## Requerimientos

Usted debe:

* Determinar cuáles de las variables consideradas son relevantes para el problema.


* Determinar si hay alguna transformación de las variables de entrada o de salida que mejore el pronóstico del modelo.


* Construir un modelo de regresión lineal que sirva como base para construir un modelo de redes neuronales artificiales.



* Construir un modelo de redes neuronales artificiales. Asimismo, debe determinar el número de neuronas en la capa o capas ocultas.


* Utiizar una técnica como crossvalidation u otra similar para establecer la robustez del modelo.

# Respuesta:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Lectura de datos

In [ ]:
diabetes = pd.read_csv('datos/diabetes.csv')

Ahora que los datos han sido leidos necesitamos saber que variables son significativas y cuales no, para lo cual debemos hacer una correlacion entre las variables.

In [ ]:
corr = diabetes.corr()
corr

Ahora veamos la correlación de manera grafica

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(corr,cmap='coolwarm', vmin=-1,vmax =1)
fig.colorbar(cax)
ticks = np.arange(0,len(diabetes.columns),1)
ax.set_xticks(ticks)
plt.xticks(rotation=90)
ax.set_yticks(ticks)
ax.set_xticklabels(diabetes.columns)
ax.set_yticklabels(diabetes.columns)
plt.show()

Tanto en la grafica como en la tabla podemos ver que las variables s1 y s2 estan altamente correlacionadas, por lo cual podemos eliminar una de estas sin perder demasiada información.

Para determinar cual eliminar hacemos un grafico de importancia usando la recursive feature elimination.

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

diabetes_data = diabetes.drop(['Y'],axis=1)
diabetes_target = diabetes[['Y']]

model = LogisticRegression()
rfe = RFE(model,9)
rfe = rfe.fit(diabetes_data,diabetes_target)

plt_importances = pd.Series(rfe.ranking_, index = diabetes_data.columns)
plt_importances.nlargest(10).plot(kind='barh')
plt.show()

En la grafica anterior podemos ver que la variable s2 es la menos importante de todas las variables por lo cual sera esta la que salga del modelo.

In [ ]:
diabetes_data = diabetes.drop(['s2','Y'],axis=1)
diabetes_data.head()

## Modelo lineal
Ya con el dataset depurado, procedemos a hacer el modelo de regresión linear, adicional a este veremos si al aplicar alguna transformacion lineal el modelo se optimiza

### Sin transformación

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import metrics

data_train, data_test, target_train, target_test = train_test_split(diabetes_data,diabetes_target, test_size=0.30)

linearModel = linear_model.LinearRegression()
linearModel.fit(data_train,target_train)

Linear_predictions = linearModel.predict(data_test)
print('Metricas modelo sin transfromacion')
print('Error absoluto de la media:', metrics.mean_absolute_error(target_test,Linear_predictions))
print('Error cuadratico de la media:', metrics.mean_squared_error(target_test,Linear_predictions))
print('Error cuadratico de la respuesta:', np.sqrt(metrics.mean_squared_error(target_test,Linear_predictions)))

### Transformación cuadratica

In [ ]:
squareTransform = lambda x:x**2

dataFrameTransformed = diabetes_data.apply(squareTransform)
targetTransformed = diabetes_target.apply(squareTransform)

data_train, data_test, target_train, target_test = train_test_split(dataFrameTransformed,targetTransformed, test_size=0.30)

linearModel = linear_model.LinearRegression()
model = linearModel.fit(data_train,target_train)

predictions = linearModel.predict(data_test)
print('Metricas modelo con transfromacion')
print('Error absoluto de la media:', metrics.mean_absolute_error(target_test,predictions))
print('Error cuadratico de la media:', metrics.mean_squared_error(target_test,predictions))
print('Error cuadratico de la respuesta:', np.sqrt(metrics.mean_squared_error(target_test,predictions)))

### Transformación cúbica

In [ ]:
cubicTransform = lambda x:x**3

dataFrameTransformed = diabetes_data.apply(cubicTransform)
targetTransformed = diabetes_target.apply(cubicTransform)

data_train, data_test, target_train, target_test = train_test_split(dataFrameTransformed,targetTransformed, test_size=0.30)

linearModel = linear_model.LinearRegression()
model = linearModel.fit(data_train,target_train)

predictions = linearModel.predict(data_test)
print('Metricas modelo con transfromacion')
print('Error absoluto de la media:', metrics.mean_absolute_error(target_test,predictions))
print('Error cuadratico de la media:', metrics.mean_squared_error(target_test,predictions))
print('Error cuadratico de la respuesta:', np.sqrt(metrics.mean_squared_error(target_test,predictions)))

### Transformación exponencial

In [ ]:
exponentialTransform = lambda x:np.exp(x)

dataFrameTransformed = diabetes_data.apply(exponentialTransform)
targetTransformed = diabetes_target.apply(exponentialTransform)

data_train, data_test, target_train, target_test = train_test_split(dataFrameTransformed,targetTransformed, test_size=0.30)

linearModel = linear_model.LinearRegression()
model = linearModel.fit(data_train,target_train)

predictions = linearModel.predict(data_test)
print('Metricas modelo con transfromacion')
print('Error absoluto de la media:', metrics.mean_absolute_error(target_test,predictions))
print('Error cuadratico de la media:', metrics.mean_squared_error(target_test,predictions))
print('Error cuadratico de la respuesta:', np.sqrt(metrics.mean_squared_error(target_test,predictions)))

### Resultado:

## Red neuronal

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

data_train, data_test, target_train, target_test = train_test_split(diabetes_data,diabetes_target, test_size=0.30)

history_nn = []

neurons = [2,4,8,16,32,64]

for h in neurons:

    ## crea el modelo vacio
    model = tf.keras.Sequential()

    ## capa oculta
    model.add(tf.keras.layers.Dense(h,                         # neuronas en la capa oculta
                                    activation=tf.nn.relu,  # función de activación
                                    input_shape = (9,)))       # una neurona de entrada

    ## capa de salida
    model.add(tf.keras.layers.Dense(1,                         # una neurona de salida
                                    activation='linear')) # capa de salida

    ## optimizacion usando gradiente
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

    ## crea el modelo
    model.compile(optimizer=optimizer,     # optimizador
                  loss='mse',              # función a minimizar
                  metrics=['mse'])  # metricas a reportar

    ## entrena el modelo y guarda
    ## el reporte de entrenamiento
    model.fit(data_train, target_train, epochs=5000, verbose=0)

    ## error de entrenamiento
    y = [u[0] for u in model.predict(data_test).tolist()]
    history_nn.append(metrics.mean_squared_error(target_test,y))

In [ ]:
plt.figure(figsize=(10,5))
plt.xlabel('Neuronas')
plt.plot(neurons, history_nn, 'ko-')
plt.grid()

Se estima que con 7 u 8 neuronas el modelo puede hacer una prediccion acertada

### Comparacion resultados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

diabetes = pd.read_csv('datos/diabetes.csv')

diabetes_data = diabetes.drop(['s2','Y'],axis=1)
diabetes_target = diabetes[['Y']]
data_train, data_test, target_train, target_test = train_test_split(diabetes_data,diabetes_target, test_size=0.30)

from sklearn import linear_model
from sklearn import metrics

linearModel = linear_model.LinearRegression()
linearModel.fit(data_train,target_train)

linearPredictions = linearModel.predict(data_test)

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
## crea el modelo vacio
model = tf.keras.Sequential()

## capa oculta
model.add(tf.keras.layers.Dense(8,                   # neuronas en la capa oculta
                                activation=tf.nn.relu, # función de activación
                                input_shape = (9,)))    # una neurona de entrada

## capa de salida
model.add(tf.keras.layers.Dense(1,                   # una neurona de salida
                                activation='linear')) # capa de salida

## optimizacion usando gradiente
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

## crea el modelo
model.compile(optimizer=optimizer,     # optimizador
                loss='mse',              # función a minimizar
                metrics=['mse'])  # metricas a reportar

## entrena el modelo y guarda
## el reporte de entrenamiento
model.fit(data_train, target_train, epochs=5000, verbose=0)

neuralNetworkPredictions = model.predict(data_test)

In [ ]:
print('sse linear:',metrics.mean_squared_error(target_test,linearPredictions))
print('sse NN:',metrics.mean_squared_error(target_test,neuralNetworkPredictions))